In [1]:
from IPython.display import display
import torch

# Aggregation over dimensions

In [140]:
# mat is m x n matrix
mat = \
[[1,2,3],
 [4,5,6],
 [7,8,9],
 [10,11,12]]
t = torch.Tensor(mat)
i = 1 # i is rows, (i in [m])
j = 2 # j is columns (j in [n])

display(mat[i][j])
display(t[i][j])

6

tensor(6.)

![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*jmXqsVUNaBaUsBAkHgqb3A.png)


In [148]:
display(t.shape)
# sum over rows, iterate over i's \sigma_{i=1}^m t_{ij}
# [1+4+7+10, 2+5+8+11, 3+6+9+12] = [22, 26, 30]
display(t.sum(dim=0).shape) # we "lost" the 0th dimension
# sum over columns, iterate over j's \sigma_{j=1}^n t_{ij}
# [1+2+3 , 4+5+6, 7+8+9, 10+11+12] = [6, 15, 24, 33]
display(t.sum(dim=1).shape) # we "lost" the 1st dimension

display(torch.rand(4,5,6,7,8).sum(dim=3).shape) # we "lost" the 3rd dimension, which had size 7

torch.Size([4, 3])

torch.Size([3])

torch.Size([4])

torch.Size([4, 5, 6, 8])

Let's start with reading numpy's explanation of broadcasting: https://numpy.org/doc/stable/user/basics.broadcasting.html

General Broadcasting Rules
When operating on two arrays, NumPy compares their shapes element-wise. It starts with the trailing (i.e. rightmost) dimension and works its way left. Two dimensions are compatible when

1) they are equal, or

2) one of them is 1.

If these conditions are not met, a ValueError: operands could not be broadcast together exception is thrown, indicating that the arrays have incompatible shapes.  

Numpy's explaination is not really comprehensive enough and even reading the entire article doesn't provide enough intuition to convert "scientific formula" into simple broadcasted operations

Let's check this article:
https://towardsdatascience.com/broadcasting-in-numpy-58856f926d73

![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*h8hFhsht_xbQihRV8fzKAg.png)

Rule 1: If the two arrays differ in their number of dimensions, the shape of the one with fewer dimensions is padded with ones on its leading (left) side.

Rule 2: If the shape of the two arrays does not match in any dimension, the array with shape equal to 1 in that dimension is stretched to match the other shape.

Rule 3: If in any dimension the sizes disagree and neither is equal to 1, an error is raised.

Stretching means that the corresponding layers are replicated:  
![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*3owRZj2AolN6Ry45bOKskg.png)

In [58]:
A = torch.arange(2*3).reshape(2,3)
V = torch.arange(2).reshape(2,1)+7
U = torch.arange(3).reshape(1,3)+5
display(A.shape, A)
display(V.shape, V)
display(U.shape, U)

torch.Size([2, 3])

tensor([[0, 1, 2],
        [3, 4, 5]])

torch.Size([2, 1])

tensor([[7],
        [8]])

torch.Size([1, 3])

tensor([[5, 6, 7]])

The following tensors are compatible for broadcasting

A [2 3]  
V [2 1]   - dimension is 1 and then equals to A's  
U [1 3]   - dimension equals to A's and then is 1  

In [48]:
display(A)
display(U)
display(A+U)
display(A+V)
display(V)

tensor([[0, 1, 2],
        [3, 4, 5]])

tensor([[5, 6, 7]])

tensor([[ 5,  7,  9],
        [ 8, 10, 12]])

tensor([[ 7,  8,  9],
        [11, 12, 13]])

tensor([[7],
        [8]])

Numpy also provides a function called np.broadcast_to, which allows us to explicitly see the broadcasted shape without any computation or tensors

In [59]:
display(torch.broadcast_shapes((2,3), (2,1), (1,3)))
display(torch.broadcast_shapes(A.shape, V.shape, U.shape))

torch.Size([2, 3])

torch.Size([2, 3])

In [124]:
#Broadcast print!
def print_broadcast_shaped(*shapes):
    try:
        final_shape = tuple(torch.broadcast_shapes(*shapes))
    except:
        print("Shapes cannot be broadcast together.")
        return None
    # print the resulting broadcasted shape first
    print(f"{str(final_shape):>{len(str(final_shape))}}",'--',final_shape)
    for shape in shapes:
        left_prefix = ':, '*(len(final_shape)-len(shape))
        # we know that the shapes are broadcastable, so we can assume the tuples are equal
        right_shape = ', '.join(':' if shape[i] == 1 else str(shape[i]) for i in (range(len(shape))))
        pretty_shape = f"(, {shape[0]})" if len(shape) == 1 else str(shape)
        print(f"{pretty_shape:>{len(str(final_shape))}}",'->','('+left_prefix+right_shape+')')

print_broadcast_shaped(
    (1,),
    (2,),
    (3,2),
    (4,1,2),
    (5,4,3,2),
    )
print('----')
print_broadcast_shaped(
    (2,1),
    (1,3),
    )

(5, 4, 3, 2) -- (5, 4, 3, 2)
       (, 1) -> (:, :, :, :)
       (, 2) -> (:, :, :, 2)
      (3, 2) -> (:, :, 3, 2)
   (4, 1, 2) -> (:, 4, :, 2)
(5, 4, 3, 2) -> (5, 4, 3, 2)
----
(2, 3) -- (2, 3)
(2, 1) -> (2, :)
(1, 3) -> (:, 3)


From this pretty print out we can see the dimension the broadcasting "duplicates" our data on

In [4]:
zeros = torch.zeros(2,3)
display(zeros)
display(U)
display(zeros+U)
display(zeros+V)
display(V)

tensor([[0., 0., 0.],
        [0., 0., 0.]])

tensor([[5, 6, 7]])

tensor([[5., 6., 7.],
        [5., 6., 7.]])

tensor([[7., 7., 7.],
        [8., 8., 8.]])

tensor([[7],
        [8]])

https://www.wikiwand.com/en/Yorick_(programming_language) - origins of broadcasting?
https://www.wikiwand.com/en/Perl_Data_Language

comprehensive numpy tutorial
https://betterprogramming.pub/numpy-illustrated-the-visual-guide-to-numpy-3b1d4976de1d

In [5]:
print("A:")
display(A)
Vrange = torch.arange(2)+7
display(Vrange)
print('U:')
display(U)
print('unsqueeze(1):')
display(Vrange.unsqueeze(1))
display(A+Vrange.unsqueeze(1))

A:


tensor([[0, 1, 2],
        [3, 4, 5]])

tensor([7, 8])

U:


tensor([[5, 6, 7]])

unsqueeze(1):


tensor([[7],
        [8]])

tensor([[ 7,  8,  9],
        [11, 12, 13]])

In [6]:
print("A:")
display(A)
Urange = torch.arange(3)+5
display(Urange)
print('U:')
display(U)
print('unsqueeze(0):')
display(Urange.unsqueeze(0))
display(A+Urange.unsqueeze(0))


A:


tensor([[0, 1, 2],
        [3, 4, 5]])

tensor([5, 6, 7])

U:


tensor([[5, 6, 7]])

unsqueeze(0):


tensor([[5, 6, 7]])

tensor([[ 5,  7,  9],
        [ 8, 10, 12]])

In [35]:
# How would one reproduce the following matrix using broadcasting?
#          [ |  |  |  ]   [--U--]
# A* = A + [ V   V  V ] + [--U--]
#          [ |  |  | ]    [--U--]
# A[mxn] - matrix
# V[mx1] - col
# U[1xn] - row
import itertools


display(A.shape,V.shape,U.shape)
Astar = A + V + U
display(Astar)
print('------------------')
# For i,j in A:
#    A*[i,j] = A[i,j] + V[i] + U[j]
Astar_manual = torch.zeros_like(A)
display(A.shape,Vrange.shape,Urange.shape)
for (i,j) in itertools.product(range(A.shape[0]),range(A.shape[1])):
    # Astar_manual[i,j] = A[i,j] + V[i,:] + U[:,j] # these are already unqueezed in ':' dimension
    Astar_manual[i,j] = A[i,j] + Vrange[i] + Urange[j] # these are unqueezed in : dimension
display(Astar_manual)
# Whenever you want to to an V[i,:] (Vrange[i]) or U[:,j] (Urange[j]) you need to unsqueeze (have it be 1) the ':' dimension
# torch.Size([2, 1]) | V[i,:] | Vrange[i]
# torch.Size([1, 3]) | U[:,j] | Urange[j]

torch.Size([2, 3])

torch.Size([2, 1])

torch.Size([1, 3])

tensor([[12, 14, 16],
        [16, 18, 20]])

------------------


torch.Size([2, 3])

torch.Size([2])

torch.Size([3])

tensor([[12, 14, 16],
        [16, 18, 20]])

In [182]:
# Beyond the 2D!
# Beyond the 3D!
# Beyond the Time Dimension! - Can broadcasting be used for time series calculations?
# cumsum using broadcasting
display(torch.arange(5.).cumsum(dim=0))
t = torch.arange(6.)

# https://www.wikiwand.com/en/Summed_area_table
t[-1] = 0
strided = t.as_strided(size=(5,),stride=(1,),storage_offset=1)
display(t)
display(strided)
t += t[1]
# t +=strided
# t

tensor([ 0.,  1.,  3.,  6., 10.])

tensor([0., 1., 2., 3., 4., 0.])

tensor([1., 2., 3., 4., 0.])

RuntimeError: unsupported operation: some elements of the input tensor and the written-to tensor refer to a single memory location. Please clone() the tensor before performing the operation.

In [238]:
import numpy as np
# array = np.arange(6.)
array = np.arange(6.).reshape(2,3) + 3
# array([[3., 4., 5.],
#       [6., 7., 8.]])
s = np.lib.stride_tricks.as_strided(array, shape=(1,3)) # array([[3., 4., 5.]])
array += s
array
# array([[ 6.,  8., 10.],
#        [ 9., 11., 13.]])

array([[ 6.,  8., 10.],
       [ 9., 11., 13.]])

In [243]:
import torch
t = torch.arange(6.).reshape(2,3) + 3
# tensor([[3., 4., 5.],
        # [6., 7., 8.]])
s = t.as_strided(size=(1,3),stride=(1,1)) # tensor([[3., 4., 5.]])
t += s
t
# RuntimeError                              Traceback (most recent call last)
#       3 # tensor([[3., 4., 5.],
#       4         # [6., 7., 8.]])
#       5 s = t.as_strided(size=(1,3),stride=(1,1)) # tensor([[3., 4., 5.]])
# ----> 6 t += s
#       7 t

# RuntimeError: unsupported operation: some elements of the input tensor and the written-to tensor refer to a single memory location. Please clone() the tensor before performing the operation.

RuntimeError: unsupported operation: some elements of the input tensor and the written-to tensor refer to a single memory location. Please clone() the tensor before performing the operation.

In [236]:
array += np.lib.stride_tricks.as_strided(array, shape=(1,3))
array

array([[ 6.,  8., 10.],
       [ 9., 11., 13.]])

https://stackoverflow.com/q/56731361/12603110
In this question the guy requests a somewhat interesting task for combining matrices.
Lets investigate the problem and see if we can find a solution that uses broadcasting
```python
R = np.zeros([D,D])

for a in range(D):
  for b in range(D):
    for c in range(D):
      if 0 <= b+c-a < D:
        R[a, b+c-a] += T[a, b, c]*M[b, c]
```

In [36]:
D = 3
M = torch.arange(D*D).reshape(D,D)
T = torch.arange(D*D*D).reshape(D,D,D)

R = torch.zeros([D,D])

for a in range(D):
  for b in range(D):
    for c in range(D):
      if 0 <= b+c-a < D:
        R[a, b+c-a] += T[a, b, c]*M[b, c]
display(R)

tensor([[  0.,  10.,  56.],
        [ 46., 164., 182.],
        [272., 290., 208.]])

In [43]:
# Let's first solve the multiplication of T[a,b,c] * M[b,c]
# T[a,b,c] * M[b,c]
# T[a,b,c] * M[:,b,c]
display(T,M)
display(T[0] * M,T[1] * M,T[2] * M)
(T * M)

tensor([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8]],

        [[ 9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]])

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

tensor([[ 0,  1,  4],
        [ 9, 16, 25],
        [36, 49, 64]])

tensor([[  0,  10,  22],
        [ 36,  52,  70],
        [ 90, 112, 136]])

tensor([[  0,  19,  40],
        [ 63,  88, 115],
        [144, 175, 208]])

tensor([[[ 0,  2,  4],
         [ 6,  8, 10],
         [12, 14, 16]],

        [[ 9, 11, 13],
         [15, 17, 19],
         [21, 23, 25]],

        [[18, 20, 22],
         [24, 26, 28],
         [30, 32, 34]]])

# Caveats and gotchas


So, there’s a total of three types of vectors in NumPy: 1D arrays, 2D row vectors, and 2D column vectors. Here’s a diagram of explicit conversions between those:  
https://betterprogramming.pub/numpy-illustrated-the-visual-guide-to-numpy-3b1d4976de1d
![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*nVqN9Ha2tBO4isoGqCIsMQ.png)  
* flatten is always a copy
* ravel() is usually a view
* reshape(-1) is a view when possible
* `.shape=` is always a view   
(see “[5 ways to flatten an array](https://betterprogramming.pub/5-ways-to-flatten-an-array-in-numpy-dd6d79042139)” for details)  

By the rules of broadcasting, 1D arrays are implicitly interpreted as 2D row vectors, so it is generally not necessary to convert between those two — thus the corresponding area is shaded.


# Further Reading
einsum: https://betterprogramming.pub/einsum-visualized-c050903145ef